In [6]:
import gc
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import seaborn as sns
from tqdm import tqdm
import lightgbm as lgb


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import xgboost as xgb

In [8]:
df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')
target_train = df_train['y'].values
id_test = df_test['id'].values
cols_to_drop = [
    'id',
    'y',
]


In [9]:
#dummies = ["var9", "var18", "var22", "var23", "var24", "var25", "var27", "var29", "var30", "var31", "var33", "var39", "var41", "var44", "var47", "var48", "var49", "var50", "var51", "var53", "var54"]
#
#for col in dummies: # 100
#    df_train = pd.get_dummies(df_train, columns=[col])
#    df_test = pd.get_dummies(df_test, columns=[col])
#for col in df_test.columns:
#    
#    if col not in df_train.columns:
#        df_train[col] = 0
        

In [10]:
dummies = 20

for col in df_test.columns: # 100
    if (len(df_test[col].unique()) < dummies) and (len(df_test[col].unique()) == len(df_train[col].unique())):
        df_train = pd.get_dummies(df_train, columns=[col])
        df_test = pd.get_dummies(df_test, columns=[col])

In [11]:
train = np.array(df_train.drop(['y','id'], axis = 1))
test = np.array(df_test.drop(['id'], axis = 1))

xgb_preds = []

In [20]:

lgb_preds = []

In [12]:
K = 5
kf = KFold(n_splits = K, random_state = 3228, shuffle = True)

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'max_depth': 5,
    'learning_rate': 0.01, 
    'verbose': -1,
    'num_threads': 2,
}


In [47]:
xgb_w = 8
lgb_w = 10 - xgb_w

In [48]:
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
def xgb_f1(y, t, threshold=0.5):
    t = t.get_label()
    y_bin = (y > threshold).astype(int) # works for both type(y) == <class 'numpy.ndarray'> and type(y) == <class 'pandas.core.series.Series'>
    return 'f1',f1_score(t,y_bin)
err = 0
err2 = 0
err3 = 0
err4 = 0
err5 = 0
errLL = 0
cutPoint = 0.36
for train_index, test_index in kf.split(train):
    train_X, valid_X = train[train_index], train[test_index]
    train_y, valid_y = target_train[train_index], target_train[test_index]
    lgb_train = lgb.Dataset(
        train_X, 
        train_y, 
        )
    lgb_train.raw_data = None

    lgb_valid = lgb.Dataset(
        valid_X, 
        valid_y,
        )
    lgb_valid.raw_data = None

    lgb_model = lgb.train(
        
        lgb_params,
        lgb_train,
        num_boost_round=100000,
        valid_sets=[lgb_train, lgb_valid],
        early_stopping_rounds=150, 
        verbose_eval=100, 
    )
    # params configuration also from the1owl's kernel
    # https://www.kaggle.com/the1owl/forza-baseline
    xgb_params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 'objective': 'binary:logistic', 'eval_metric': 'auc', 'seed': 99, 'silent': True}
    #xgb_params = {'eta': 0.11, 'max_depth': 5, 'subsample': 0.4, 'colsample_bytree': 0.4, 'objective': 'binary:logistic', 'min_child_weight': 15, 'eval_metric': 'auc', 'seed': 99, 'silent': True}

    d_train = xgb.DMatrix(train_X, train_y)
    d_valid = xgb.DMatrix(valid_X, valid_y)
    d_test = xgb.DMatrix(test)
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    model = xgb.train(xgb_params, d_train, 5000,  watchlist, maximize=True, verbose_eval=50, early_stopping_rounds=100)
                        
    xgb_pred = model.predict(d_test)
    lgb_pred = lgb_model.predict(test)
    pG = model.predict(d_valid)

    errLL += log_loss(valid_y, pG)
    p = pG
    p[p > cutPoint] = 1
    p[p != 1] = 0
    err += f1_score(valid_y, p)
    p = pG
    p[p > (cutPoint+0.01)] = 1
    p[p != 1] = 0
    err2 += f1_score(valid_y, p)
    p = pG
    p[p > (cutPoint+0.02)] = 1
    p[p != 1] = 0
    err3 += f1_score(valid_y, p)
    p = pG
    p[p > (cutPoint-0.02)] = 1
    p[p != 1] = 0
    err4 += f1_score(valid_y, p)
    p = pG
    p[p > (cutPoint-0.01)] = 1
    p[p != 1] = 0
    err5 += f1_score(valid_y, p)
    print(err)
    xgb_preds.append(list((((xgb_pred*xgb_w)+(lgb_pred*lgb_w)))/10))

Training until validation scores don't improve for 150 rounds
[100]	training's binary_logloss: 0.350094	valid_1's binary_logloss: 0.364902
[200]	training's binary_logloss: 0.310633	valid_1's binary_logloss: 0.337318
[300]	training's binary_logloss: 0.288855	valid_1's binary_logloss: 0.325851
[400]	training's binary_logloss: 0.272348	valid_1's binary_logloss: 0.319852
[500]	training's binary_logloss: 0.259674	valid_1's binary_logloss: 0.317001
[600]	training's binary_logloss: 0.248894	valid_1's binary_logloss: 0.315123
[700]	training's binary_logloss: 0.23886	valid_1's binary_logloss: 0.314268
[800]	training's binary_logloss: 0.230239	valid_1's binary_logloss: 0.313626
[900]	training's binary_logloss: 0.222525	valid_1's binary_logloss: 0.313632
[1000]	training's binary_logloss: 0.215433	valid_1's binary_logloss: 0.313432
[1100]	training's binary_logloss: 0.208511	valid_1's binary_logloss: 0.313298
[1200]	training's binary_logloss: 0.20222	valid_1's binary_logloss: 0.313299
[1300]	traini

In [45]:
xgb_pred

array([0.8809537 , 0.5762347 , 0.22421467, ..., 0.5541503 , 0.4149281 ,
       0.14109486], dtype=float32)

In [46]:
print(err/5) #print(err/4) #6711266
print(err2/5) #print(err/4) #6711266
print(err3/5) #print(err/4) #6711266
print(err4/5) #print(err/4) #6711266
print(err5/5) #print(err/4) #6711266
print(errLL/5) #print(err/4) #6711266

0.6713119799988848
0.6713119799988848
0.6713119799988848
0.6713119799988848
0.6713119799988848
0.307419797130156


In [49]:
preds=[]
for i in range(len(xgb_preds[0])):
    sum=0
    for j in range(K):
        sum+= xgb_preds[j][i]
    preds.append(sum / K)

output = pd.DataFrame({'id': id_test, 'predicted': preds})
output['predicted'][output['predicted'] > cutPoint] = 1
output['predicted'][output['predicted'] != 1] = 0
output['predicted'] = output['predicted'].astype(int)
output.to_csv("../data/output/{}-foldCV_avg_sub_dummy_person_cutpoint{}_error{}_logloss{}_stackingXGBLGB_w{}.csv".format(K,cutPoint, err/5, errLL/5, xgb_w), index=False)   

<ipython-input-49-f6a6dd67e39b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predicted'][output['predicted'] > cutPoint] = 1
<ipython-input-49-f6a6dd67e39b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output['predicted'][output['predicted'] != 1] = 0


In [33]:
preds

[1.8042323565804277,
 0.41038777477949206,
 -1.0522606733771749,
 -0.25138078858893376,
 -0.4567954260273539,
 -2.2922542963187715,
 0.33318610444021457,
 -3.3716836687302,
 2.552722908285543,
 -3.4688423788692537,
 -0.4455704650340236,
 1.2797088465732864,
 0.5514737085645274,
 -1.2745198788317063,
 -0.6736042820644619,
 -4.10655505313605,
 -4.538364533304628,
 -2.5795617502397694,
 2.6183872462242244,
 -0.2533613574804656,
 -3.0643439596828848,
 -1.6787980568611176,
 -1.8698275895147318,
 2.3225378825904186,
 -0.7865530631017743,
 -2.578908246508007,
 -1.6735292449659993,
 -1.8892818975977228,
 0.6465017610200811,
 0.9993018340232499,
 0.3842278990387601,
 -3.5969435918457213,
 -2.4293568463253603,
 0.9594646578806941,
 -0.31569421708026846,
 1.9429804954573477,
 -0.5364576101594073,
 -1.5985411919160997,
 -0.0631660058926542,
 1.0613999322658283,
 1.3967211891082578,
 -2.4080408322920044,
 -2.1244810245999863,
 -3.6158205747693897,
 0.7666878120819897,
 -1.0635515485839295,
 -3.1024

In [17]:
lgb_preds

[]

In [99]:
sub = pd.read_csv("./5-foldCV_avg_sub_36_dummy.csv")

In [100]:
sub["v"] = output["predicted"]

In [101]:
sub["v2"] = sub["v"] - sub["predicted"]

In [102]:
sub[sub["v2"] != 0]

,id,predicted,v,v2


In [84]:
119

152